# Tutorial: Converting a pandapowerNet to PSS Sincal - A minial Example

This interactive Jupyter Notebook tutorials demonstrates how a small **pandapowerNet** can be created and afterwards be converted to **PSS Sincal**. The three main steps **initialize**, **convert** and **finalize** are beein executed. An installation of PSS Sincal is needed to finnish the converting, also the needed python packages must be installed in your environment.

Import all needed python-packages that are used in the following code.

In [ ]:
import os

from pandapower.test import test_path
from pandapower.converter.sincal.pp2sincal.util.main import initialize, convert_pandapower_net, finalize
from pandapower.converter.sincal.pp2sincal.pp2sincal import pp2sincal

from pandapower.plotting import simple_plot

import pandapower as pp

We imported all packages. Now we create a small network. The network is part of the minimal example tutorial from the pandapower repository: https://github.com/e2nIEE/pandapower/blob/master/tutorials/minimal_example.ipynb

In [ ]:
# create empty net
net_pp = pp.create_empty_network()

# create buses
bus1 = pp.create_bus(net_pp, vn_kv=20., name="Bus 1")
bus2 = pp.create_bus(net_pp, vn_kv=0.4, name="Bus 2")
bus3 = pp.create_bus(net_pp, vn_kv=0.4, name="Bus 3")

# create bus elements
pp.create_ext_grid(net_pp, bus=bus1, vm_pu=1.02, name="Grid Connection")
pp.create_load(net_pp, bus=bus3, p_mw=0.100, q_mvar=0.05, name="Load")

# create branch elements
trafo = pp.create_transformer(net_pp, hv_bus=bus1, lv_bus=bus2, std_type="0.4 MVA 20/0.4 kV", name="Trafo")
line = pp.create_line(net_pp, from_bus=bus2, to_bus=bus3, length_km=0.1, std_type="NAYY 4x50 SE", name="Line")

We can look at the dataset by adressing the columns of the **pandas DataFrames**.

In [ ]:
net_pp.bus

Running a loadflow gives us informations in the net_pp.res_ columns.

In [ ]:
pp.runpp(net_pp)
net_pp.res_bus

In [ ]:
net_pp.res_line

Using the **simple_plot()** method from the plotting module out of pandapower, gives us an overview of the net topology. Due to the fact that we didn't define any geocoordinates, the method will generate **generic** coordinates base on a **networkx graph**. This procedure takes also place in the pp2sincal converter. The graphical aquivalent of the network will be displayed on the are tile in sincal if the plotting flag is set to True.

In [ ]:
# simple plot of the network
simple_plot(net_pp)

We erase the already generated coordinates in order to test the converter with a network without any coordinates.

In [ ]:
# Drop generic coordinates 
net_pp.bus_geodata.drop(index=net_pp.bus.index, inplace=True)

To convert our small network to sincal we have to define an output path and some flags for the used methods from the pp2sincal-converter. In this case we use the already imported test_path from the converters repository.

In [ ]:
# Set Output path
output_folder = os.path.join(test_path, 'converter', 'test_to_sincal', 'results', 'minimal_example')
file_name = 'minimal_example_wo_main' + '.sin'
use_active_net = False
use_ui = False
sincal_interaction = False
plotting = True
delete_files = True

The next steps involve the three main function.

In [ ]:
# 1. Initialization
net, app, sim, doc = initialize(output_folder, file_name, use_active_net, use_ui, sincal_interaction, delete_files)

# 2. Conversion
convert_pandapower_net(net, net_pp, doc, plotting)

# 3. Finalization
finalize(net, output_folder, file_name, app, sim, doc, sincal_interaction)

The .sin file of the network is saved at the test_path. 

As an alternative to the shown procedure, we can directly convert the network by using the main method. We change the file_name and convert again. But first of all, we have to drop already generated generic coordinates from the first conversion.

In [ ]:
# Drop already generated generic coords
net_pp.bus_geodata.drop(index=net_pp.bus.index, inplace=True)

In [ ]:
file_name = 'minimal_example_with_main' + '.sin'

In [ ]:
# main method combines the three steps initialize, convert and finalize
pp2sincal(net_pp, output_folder, file_name, use_active_net, plotting, sincal_interaction)